# Data Mining Challenge 2020

## Team
* Tommaso Fioravanti - 105xxxxx(Student-ID)
* Matteo Moreschini - 105xxxxx(Student-ID)
* Alessio Introito Russo - 105xxxxx(Student-ID)
* Davide Salaorni - 10526800


# 1. Introduction
Questo progetto serve a...

# 2. Data Analysis

# 3. Baseline

# 4. Cluster Analysis

# 5. Feature Engineering

# 6. Feature Selection

# 7. Model

# 8. Feature Importance

# 9. Model Comparison

# 10. Error Analysis

# 11. Future Steps (?)